In [1]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    from dolfin import *

try:
  import gmsh
except ImportError:
  !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
  import gmsh

import os
import numpy as np
import matplotlib.pyplot as plt

--2024-12-01 12:33:52--  https://fem-on-colab.github.io/releases/fenics-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4164 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.07K  --.-KB/s    in 0s      

2024-12-01 12:33:52 (44.1 MB/s) - ‘/tmp/fenics-install.sh’ saved [4164/4164]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/1fa0ca14/releases/pybind11-ins

In [2]:
!gmsh -2 -format msh2 simple_sym.geo
!dolfin-convert simple_sym.msh lab.xml

Info    : Running 'gmsh -2 -format msh2 simple_sym.geo' [Gmsh 4.14.0-git-dd9cf194a, 1 node, max. 1 thread]
Info    : Started on Sun Dec  1 12:36:05 2024
Info    : Reading 'simple_sym.geo'...
Info    : Done reading 'simple_sym.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 80%] Meshing curve 14 (Line)
Info    : [ 90%] Meshing curve 15 (Line)
Info    : [100%] Meshing curve 16 (Line)
Info    : Done meshing 1D (Wall 0.00907466s, CPU 0.006341s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay for

In [3]:
import numpy as np
from sys import argv

# Parámetros
meshName = "lab"
caseDir = "S12"
saveSteps = 10
aspect_hl = 4 # Rel aspecto long. transicion vs altura minima de elemento
l = aspect_hl*7.5e-4 # Longitud de transición

# Material
E = 2450
nu = 0.3
Gc = 0.1 # Energía por unidad de superficie de fractura
g = 9.81
densidad = 1200 * 0.035 # kg/m3 * h = km/m2

# Fzas exteriores
px = 10.
py = 0.

# Condiciones iniciales
w_init = 5e-4
l_init = 0.01
p_init = 10
center = [0.0, 0.0]

# Tolerancias
t = 0
Tfinal = 400
delta_p = 0.1
deltaT = 0.1
tolphi = 1e-3
toldispl = 1e-6
step = 0

## aumentar presión si converge rápido
MAXITE = 5
deltap_counter = 0
DELTAP_AUG = 5

## MESHING ##
mesh = Mesh(meshName+ ".xml")
subdomains = MeshFunction('size_t',mesh,meshName+"_physical_region.xml")
boundary_markers = MeshFunction('size_t',mesh, meshName+"_facet_region.xml")

# 1 . MESH TIENE QUE SER DOMAIN
V = FunctionSpace(mesh, 'CG', 1)
W = VectorFunctionSpace(mesh, 'CG', 1)

qo = 3.4 * (1e-6/60)  # 3.4 ml/min

WW = FunctionSpace(mesh, 'DG', 0)
p, q = TrialFunction(V), TestFunction(V)
u, v = TrialFunction(W), TestFunction(W)

def lame_lambda(E, nu):
    return E*nu / ((1+nu) * (1-2*nu))

def lame_mu(E, nu):
    return E / (2*(1+nu))

lmbda = lame_lambda(E, nu)
mu = lame_mu(E, nu)

def epsilon(u):
  return sym(grad(u))

def sigma(u):
  return 2.0*mu*epsilon(u)+lmbda*tr(epsilon(u))*Identity(len(u))

# Densidad de energia elastica eq (12)
def psi(u):
  return 0.5*(lmbda+mu)*(0.5*(tr(epsilon(u)) + abs(tr(epsilon(u)))))**2 + mu*inner(dev(epsilon(u)), dev(epsilon(u)))

# Parámetro de trayectoria eq (11)
def H(uold, unew, Hold):
  return conditional(lt(psi(uold), psi(unew)), psi(unew), Hold)

class CrackDomain(SubDomain):
  def inside(self, x, on_boundary):
    return abs(x[0] - center[0]) <= l_init and abs(x[1] - center[1]) <= w_init

bcright = DirichletBC(W.sub(0), Constant(0.0), boundary_markers, 10)
bciny = DirichletBC(W.sub(0), Constant(0.0), boundary_markers, 40)

# Aplicar desplazamiento en la dirección y positiva
#load = Expression("t", t=0.0, degree=1)
bcbottom = DirichletBC(W.sub(1), Constant(0.0), boundary_markers, 20)

bc_u = [bcright, bcbottom]#, bciny]

# Condicion de borde de la fractura, se marca la entalla inicial con el valor de 1
crack = CrackDomain()
bc_phi = [DirichletBC(V, Constant(1.0), crack)]

unew, uold, ut = Function(W), Function(W), Function(W, name="displacement")
pnew, pold, Hold, phit = Function(V), Function(V), Function(V), Function(V, name="phi")

# Fzas externas
p_value = p_init
pressure = Constant(p_value)
ds = Measure("ds")(subdomain_data=boundary_markers)
n = FacetNormal(mesh)

# Funcional del desplazamiento eq (x)
txx = Constant((-px, 0.0))
tyy = Constant((0.0, py))
# Funcional del desplazamiento eq (x)
E_du = (1-pold)**2*inner(epsilon(v), sigma(u))*dx + pressure * inner(v, grad(pold))*dx + inner(txx, v) * ds(30) + inner(tyy, v) * ds(50)

# Funcional phi eq (x)
E_phi = (Gc*l*inner(grad(p), grad(q)) + ((Gc/l)+2.0 *H(uold, unew, Hold))*inner(p,q)-2.0*H(uold, unew,Hold)*q)*dx


p_disp = LinearVariationalProblem(lhs(E_du), rhs(E_du), unew, bc_u)
p_phi = LinearVariationalProblem(lhs(E_phi), rhs(E_phi), pnew, bc_phi)

solver_disp = LinearVariationalSolver(p_disp)
solver_phi = LinearVariationalSolver(p_phi)
solver_disp.solve()

os.mkdir(caseDir)

file_results = XDMFFile(f"{caseDir}/{caseDir}.xdmf")
file_results.parameters["flush_output"] = True
file_results.parameters["functions_share_mesh"] = True
fname = open(f"{caseDir}/output.csv", 'w')


while t<=Tfinal:
  step += 1

  deltap_counter += 1 # Si converge sin necesidad de bajar la Presion entonces aceleramos

  if deltap_counter == DELTAP_AUG:
    deltap_counter = 0
    if delta_p < 0.1:
      delta_p = 0.1
      print("Increasing Delta P")
  ## Solve
  t += deltaT
  p_value += delta_p*p_value
  pressure.assign(p_value)

  print(f"Step: {step} Solving for t: {t:.9f} pressure: {p_value:.2f} -----")

  ite = 0
  err_u = 1
  err_phi = 1

  while (err_u > toldispl) or (err_phi > tolphi):
    ite += 1

    if ite % MAXITE == 0:
      p_value *= 0.98
      delta_p = 0.01
      deltap_counter = 0

      pressure.assign(p_value)
      print(f"Disminuir presión a P: {p_value:.2f}")
      # Resetear uold y pold
      uold.assign(ut)
      pold.assign(phit)
      Hold.assign(project(psi(uold), WW))

    solver_disp.solve()
    solver_phi.solve()
    err_u = errornorm(unew, uold, norm_type='l2', mesh=None)
    err_phi = errornorm(pnew, pold, norm_type='l2', mesh=None)
    uold.assign(unew)
    pold.assign(pnew)
    Hold.assign(project(H(uold, unew, Hold), WW))

    print(f"Ite: {ite} -- error phi: {err_phi:.2e} -- error u: {err_u:.2e}" )

  ## Converge
  ut.assign(unew)
  phit.assign(pnew)

  if step%saveSteps == 0:
    file_results.write(ut  , t)
    file_results.write(phit, t)
    print("XMDF Updated")

  fname.write(str(step) + ",")
  fname.write(str(t) + ",")
  fname.write(str(p_value) + "\n")
  #print("Reaccion", reaccion)
  print("Step Converged.")

fname.close()
print("Simulation completed")


Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_3801828c0f66b7190a7fd5819465b3d5b34b9149

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00612092 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.459264 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:----------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_4f750817ecc896f3bedcb4ff8c9f3352153b1b38

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00442743 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.016921 seconds.

INFO:FFC:Compiler stage 3

Calling FFC just-in-time (JIT) compiler, this may take some time.


...

Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_a66ac628399cef3ed97dce589d0731d17967233c

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       2
  Number of cell subdomains: 0
  Rank:                      1
  Arguments:                 '(v_0)'
  Number of coefficients:    3
  Coefficients:              '[f_23, f_26, f_38]'
  Unique elements:           'DG0(?,?), Vector<2 x CG1(?,?)>, CG1(?,?)'
  Unique sub elements:       'DG0(?,?), Vector<2 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 1
INFO:FFC:  quadrature_degree: 1
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.0434279 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FF

Ite: 1 -- error phi: 8.13e-03 -- error u: 2.58e-02
Ite: 2 -- error phi: 1.72e-04 -- error u: 4.18e-05
Ite: 3 -- error phi: 6.82e-04 -- error u: 6.70e-07
Step Converged.
Step: 2 Solving for t: 0.200000000 pressure: 12.10 -----
Ite: 1 -- error phi: 2.09e-03 -- error u: 2.02e-05
Ite: 2 -- error phi: 3.75e-04 -- error u: 2.16e-05
Ite: 3 -- error phi: 2.20e-03 -- error u: 7.99e-06
Ite: 4 -- error phi: 4.66e-04 -- error u: 2.71e-05
Disminuir presión a P: 11.86
Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_2e924d9ea045d2e07621ce4cd3c4eb7c0492550c

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       2
  Number of cell subdomains: 0
  Rank:                      1
  Arguments:                 '(v_0)'
  Number of coefficients:    1
  Coefficients:              '[f_26]'
  Unique elements:           'DG0(?,?), Vector<2 x CG1(?,?)>'
  Unique sub elements:       'DG0(?,?), Vector<2 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 0
INFO:FFC:  quadrature_degree: 0
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.0361202 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:--------------------

Streaming output truncated to the last 5000 lines.
Ite: 2 -- error phi: 8.02e-05 -- error u: 1.35e-06
Ite: 3 -- error phi: 7.40e-05 -- error u: 1.14e-06
Ite: 4 -- error phi: 7.34e-05 -- error u: 1.09e-06
Disminuir presión a P: 28.51
Ite: 5 -- error phi: 2.72e-05 -- error u: 4.47e-06
Ite: 6 -- error phi: 2.87e-05 -- error u: 8.88e-08
Step Converged.
Step: 706 Solving for t: 70.600000000 pressure: 28.80 -----
Ite: 1 -- error phi: 5.59e-05 -- error u: 5.22e-06
Ite: 2 -- error phi: 5.49e-05 -- error u: 8.45e-07
Step Converged.
Step: 707 Solving for t: 70.700000000 pressure: 29.08 -----
Ite: 1 -- error phi: 9.28e-05 -- error u: 5.89e-06
Ite: 2 -- error phi: 8.16e-05 -- error u: 1.39e-06
Ite: 3 -- error phi: 7.70e-05 -- error u: 1.19e-06
Ite: 4 -- error phi: 7.72e-05 -- error u: 1.14e-06
Disminuir presión a P: 28.50
Ite: 5 -- error phi: 2.80e-05 -- error u: 4.49e-06
Ite: 6 -- error phi: 3.18e-05 -- error u: 1.09e-07
Step Converged.
Step: 708 Solving for t: 70.800000000 pressure: 28.79 -----


KeyboardInterrupt: 

In [ ]:
!rm -rf /content/S8

In [ ]:
from google.colab import files
!zip -r /content/sample_data.zip /content/S5
files.download('/content/sample_data.zip')